In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
import random
import pandas as pd
from numpy import inf
import torch
from torch import nn
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.optim as optim
import json
import itertools
import math
import pyprind
from random import sample

In [2]:
#f = open('data_irl_model/Feb23_data_exploratory_policy.json')
f = open('data_irl_model/Feb22_dataSVO.json')
data_irl = json.load(f)

In [3]:
data_keys=list(data_irl.keys())
data_keys.sort()

keys_for_training=[]
keys_for_testing=[]

index=0
for vehicle in data_keys:
    if index%10==0:
        keys_for_testing.append(vehicle)
        index+=1
    else:
        keys_for_training.append(vehicle)
        index+=1

In [4]:
len(keys_for_training)

5400

In [ ]:
data_irl[str(keys_for_testing[10])]

In [61]:
time_step=5
[data_irl[str(keys_for_testing[0])][0],data_irl[str(keys_for_testing[0])][1]]#[:][time_step]

input_tens=torch.empty(len(data_irl[str(keys_for_testing[0])]),dtype=torch.float)#.to_device(self.device)
for i in range(len(data_irl[str(keys_for_testing[0])])):
    torch.cat((input_tens,torch.tensor(data_irl[str(keys_for_testing[0])][i][time_step])))

print(input_tens)

RuntimeError: zero-dimensional tensor (at position 1) cannot be concatenated

In [79]:
a123=np.array(data_irl[str(keys_for_testing[0])])
float_arr = np.vstack(a123[:, :]).astype(np.float)
#float_arr = np.vstack(a123[:, time_step]).astype(np.float)
#input=torch.tensor(float_arr,dtype=torch.float)
#input



C:\Users\nish\AppData\Local\Temp/ipykernel_15496/2787279764.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_arr = np.vstack(a123[:, time_step]).astype(np.float)


tensor([[-41.2827],
        [-48.9769],
        [  8.9757],
        [  6.6194],
        [  1.1115],
        [ -1.5041]])

In [70]:
#########################################################################
################################ AIS_gen ################################

################################ Model 1 ################################ 

class GenerateAis1(nn.Module):

    #def __init__(self,n_input,n_state,n_psi2_in=64,n_psi2_out=128):
    def __init__(self,n_input,n_state,n_psi2_in=8,n_psi2_out=16):
        super(GenerateAis1,self).__init__()
        self.PSI_layer1=nn.Linear(n_input,n_psi2_in)     # Use RELU after
        self.PSI_layer2=nn.Linear(n_psi2_in,n_psi2_out)      # Use RELU after
        self.PSI_layer3=nn.GRUCell(n_psi2_out,n_state)       # This is the Gated layer

    def forward(self,x,h):
        #x=torch.transpose(x,0,1)
        #h=torch.transpose(h,0,1)
        x=torch.relu(self.PSI_layer1(x))
        x=torch.relu(self.PSI_layer2(x))
        h=self.PSI_layer3(x,h)
        return h 
    
################################ Model 2 ################################ 

class GenerateAis2(nn.Module):

    #def __init__(self,n_input,n_state,n_psi2_in=64,n_psi2_out=128):
    def __init__(self,n_input,n_state_enc,n_psi1_out=8):
        super(GenerateAis2,self).__init__()
        self.PSI_layer1=nn.Linear(n_input,n_psi1_out)     # Use RELU after
        self.PSI_layer2=nn.GRUCell(n_psi1_out,n_state_enc)       # This is the Gated layer

    def forward(self,x,h):
        #x=torch.transpose(x,0,1)
        #h=torch.transpose(h,0,1)
        x=torch.relu(self.PSI_layer1(x))
        h=self.PSI_layer2(x,h)
        return h 

In [71]:
#########################################################################
################################ AIS_pred ###############################

################################ Model 1 ################################ 

class PredictAis1(nn.Module):
    
    #def __init__(self,n_output,n_state,n_phi2_in=16,n_phi2_out=8):
    def __init__(self,n_output,n_state,n_phi2_in=32,n_phi2_out=64):
        super(PredictAis1,self).__init__()
        self.PHI_layer1=nn.Linear(n_state,n_phi2_in)  # Use RELU after
        self.PHI_layer2=nn.Linear(n_phi2_in,n_phi2_out)     # Use RELU after
        self.PHI_layer3=nn.Linear(n_phi2_out,n_output)         # mean vector of a unit-variance multivariate Gaussian distribution, samples from which are used to predict the next observation

    def forward(self,x):
        x=torch.relu(self.PHI_layer1(x))
        x=torch.relu(self.PHI_layer2(x))
        output=self.PHI_layer3(x)
        return output


################################ Model 2 - RNN based ################################ 

class PredictAis2(nn.Module):
    
    #def __init__(self,n_output,n_state,n_phi2_in=16,n_phi2_out=8):
    def __init__(self,n_output,n_input_dec,n_state_dec):
        super(PredictAis2,self).__init__()
        #self.PHI_layer1=nn.Linear(n_output,n_state_enc)  # Use RELU after
        self.PHI_layer2=nn.GRUCell(n_input_dec,n_state_dec)       # This is the Gated layer - n_state_enc+1 for time step as input
        self.PHI_layer3=nn.Linear(n_state_dec,n_output)         # mean vector of a unit-variance multivariate Gaussian distribution, samples from which are used to predict the next observation

    def forward(self,x,h):        
        #x=self.PHI_layer1(x)   
        h=self.PHI_layer2(x,h)
        output=self.PHI_layer3(h)
        return output,h




In [104]:
class NN_structure(object):
    def __init__(self,n_epochs,min_timesteps,n_rollout,n_test,n_input,n_output,n_state_enc,n_state_dec,learning_rate,ais_gen_model,ais_pred_model,device):
        self.n_epochs=n_epochs
        self.min_timesteps=min_timesteps
        self.rollout=n_rollout
        self.n_test=n_test
        self.n_input=n_input
        self.n_output=n_output
        self.n_state_enc=n_state_enc    # Hidden state size in RNN
        self.n_state_dec=n_state_dec
        self.learning_rate=learning_rate                
        self.device=device
        self.ais_gen_model=ais_gen_model
        self.ais_pred_model=ais_pred_model

        self.n_psi2_in=8
        self.n_psi2_out=16
        self.n_phi2_in=32
        self.n_phi2_out=64
        self.n_psi1_out=8
        
        if ais_gen_model==1:
            self.gen_model=GenerateAis1(self.n_input,self.n_state_enc,self.n_psi2_in,self.n_psi2_out).to(self.device)      
        
        if ais_gen_model==2: 
            self.gen_model=GenerateAis2(self.n_input,self.n_state_enc,self.n_psi1_out).to(self.device)  
        
        if ais_pred_model==1:
            self.pred_model=PredictAis1(self.n_output,self.n_state_enc,self.n_phi2_in,self.n_phi2_out).to(self.device)
        
        if ais_pred_model==2:
            self.pred_model=PredictAis2(self.n_output,self.n_state_enc+1,self.n_state_dec).to(self.device)

    def save_model_weights(self,text="_"):
        #name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        
        name_gen_path="_"
        name_pred_path="_"
        if self.ais_pred_model==1:
            name_pred_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
            name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"

        if self.ais_pred_model==2:
            name_pred_path="trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
            name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"

        torch.save(self.gen_model.state_dict(),name_gen_path)
        torch.save(self.pred_model.state_dict(),name_pred_path)

        return "saved"
    
    def load_model_weights(self,text="_"):
        #name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        #self.gen_model.load_state_dict(torch.load(name_gen_path))
        #self.gen_model.eval()

        name_gen_path="_"
        name_pred_path="_"
        
        if self.ais_pred_model==1:
            name_pred_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
            name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"

        if self.ais_pred_model==2:
            name_pred_path="trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
            name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
        
        #torch.save(self.pred_model.state_dict(),name_pred_path)
        self.gen_model.load_state_dict(torch.load(name_gen_path))
        self.gen_model.eval()

        self.pred_model.load_state_dict(torch.load(name_pred_path))
        self.pred_model.eval()

        return "loaded"
    

    def train(self):
        train_bar=pyprind.ProgBar(self.n_epochs)
        #self.optimizer_enc = torch.optim.Adam(list(self.gen_model.parameters()) + list(self.pred_model.parameters()), lr=self.learning_rate, amsgrad=True)
        #self.optimizer_dec = torch.optim.Adam(list(self.pred_model.parameters()), lr=self.learning_rate, amsgrad=True)
        self.optimizer = torch.optim.Adam(list(self.gen_model.parameters()) + list(self.pred_model.parameters()), lr=self.learning_rate, amsgrad=True)
        self.loss=nn.MSELoss()
        self.loss_train=[]
        self.loss_train.append(0)

        for epoch in range(self.n_epochs):

            train_bar.update()
            keys_training=sample(keys_for_training,len(keys_for_training)) #shuffle the keys
            #train_bar=pyprind.ProgBar(len(keys_for_training))

            for key in keys_training:
                #train_bar.update()
                key=str(key)
                key_loss=0
                ############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps######
                negative_vals=[]
                for i in [0,1]:
                    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
                num_timesteps=min(self.min_timesteps,len(negative_vals[0]),len(negative_vals[1]))

                # Ais initialization
                time_step_loss=0

                ais=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device)

                action_list=[0,0] ## initialize the action list differently ?????                

                #for time_step in range(len(data_irl[key][0])-1):
                    
                for time_step in range(num_timesteps):                    
                    observation_list=[]
                    for i in range(len(data_irl[key])-2): # exclude the last two columns (actions/inputs)
                        temp=data_irl[key][i][time_step]
                        if temp!= None:
                            observation_list.append(temp)
                        
                    input_list=[]
                    input_list.extend(observation_list)
                    input_list.extend(action_list)
                    input_tensor=torch.tensor(input_list,dtype=torch.float).to(self.device)
                    '''
                    print("\n")
                    print("Before rollout, the input is ",input_tensor)
                    '''

                    # Update action list for the next time step and for the reference
                    for i in [4,5]:
                        temp=data_irl[key][i][time_step]
                        if temp!= None:
                            action_list[i-4]=temp
                        else:
                            action_list[i-4]=0

                    reference_list=[]
                    observation_list_ref=[]
                    for i in [1,3]:#range(len(data_irl[key])-2): # exclude the last two columns (actions/inputs)
                        temp=data_irl[key][i][time_step+1]
                        if temp!= None:
                            observation_list_ref.append(temp)                      
                    reference_list.extend(observation_list_ref)
                    '''
                    print(action_list)
                    '''
                    reference_list.extend([action_list[1]])
   
                    reference_tensor=torch.tensor(reference_list,dtype=torch.float).to(self.device)
                    '''
                    print("Before rollout, the reference is ",reference_tensor)
                    '''
                    #############################
                    # Setting things up for the rollout
                    ais_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device) # Just initialize it to 0.5
                    ais_rollout_dec=0.5*torch.ones(self.n_state_dec,dtype=torch.float).to(self.device) # Just initialize it to 0.5                   


                    rollout_prediction=0.5*torch.ones(self.n_output,dtype=torch.float).to(self.device)


                    internal_rollout_horizon=min(self.rollout,len(data_irl[key][0])-time_step-2)
                    rollout_loss=0
                    # Rollout the rnn only for 10 more time steps from time_step to time_step+rollout_horizon
                    
                    for k in range(internal_rollout_horizon):
                        rollout_reference_list=[]

                        if k==0:
                            rollout_input=input_tensor
                            rollout_reference=reference_tensor
                            '''
                            print("for rollout",k,"rollout_reference is ",rollout_reference)
                            '''
                            ais_rollout_enc=ais
                            ais_rollout_enc=self.gen_model(rollout_input,ais_rollout_enc)
                            ais=ais_rollout_enc
                            
                            #decoder_input=list(ais_rollout_enc.detach().cpu().numpy())
                            #decoder_input.extend([time_step+k])
                            #decoder_input_tensor=torch.tensor(decoder_input,dtype=torch.float).to(self.device)

                            time_step_tensor=torch.tensor([time_step+k],dtype=torch.float).to(self.device)
                            decoder_input_tensor=torch.cat((ais_rollout_enc,time_step_tensor),0)

                            rollout_prediction,ais_rollout_dec=self.pred_model(decoder_input_tensor,ais_rollout_dec)
                            '''
                            print("for rollout",k,"decoder_input  is ",decoder_input_tensor)
                            print("for rollout",k,"rollout_prediction is ",rollout_prediction)
                            '''

                            rollout_loss+=-torch.distributions.MultivariateNormal(rollout_prediction,torch.eye(rollout_prediction.shape[0]).to(self.device)).log_prob(rollout_reference)

                            #key_loss+=-torch.distributions.MultivariateNormal(rollout_prediction,torch.eye(rollout_prediction.shape[0]).to(self.device)).log_prob(rollout_reference)

                        else:                           
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i<4:
                                    rollout_reference_list.append(data_irl[key][i][time_step+k+1])
                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    rollout_reference_list.append(data_irl[key][i][time_step+k])
                                else:
                                    rollout_reference_list.append(0) # Zero padding for the input action

                            rollout_reference=torch.tensor(rollout_reference_list,dtype=torch.float).to(self.device)
                            '''
                            print("for rollout",k,"rollout_reference is ",rollout_reference)
                            '''

                            #decoder_input=list(ais_rollout_enc.detach().cpu().numpy())
                            #decoder_input.extend([time_step+k])
                            #decoder_input_tensor=torch.tensor(decoder_input,dtype=torch.float).to(self.device)

                            time_step_tensor=torch.tensor([time_step+k],dtype=torch.float).to(self.device)
                            decoder_input_tensor=torch.cat((ais_rollout_enc,time_step_tensor),0)

                            rollout_prediction,ais_rollout_dec=self.pred_model(decoder_input_tensor,ais_rollout_dec)
                            '''
                            print("for rollout",k,"decoder_input  is ",decoder_input_tensor)
                            print("for rollout",k,"rollout_prediction is ",rollout_prediction)
                            '''

                            rollout_loss+=-torch.distributions.MultivariateNormal(rollout_prediction,torch.eye(rollout_prediction.shape[0]).to(self.device)).log_prob(rollout_reference)

                    '''
                    self.optimizer_dec.zero_grad()
                    rollout_loss.backward()
                    self.optimizer_dec.step()
                    '''
                    time_step_loss+=rollout_loss
                
                self.optimizer.zero_grad()
                time_step_loss.backward()
                self.optimizer.step()

                #self.optimizer_enc.zero_grad()
                #key_loss.backward()
                #self.optimizer_enc.step()
                #print("saving model weights")
                #status=self.save_model_weights("Rollout_10")               
            print("saving model weights for epoch ",epoch)
            status=self.save_model_weights("reduced_timesteps10_Rollout_5_added_time_step_to_decoder_input")



    def test(self,test_keys):
        test_bar=pyprind.ProgBar(len(test_keys))
        self.comparision_list=[]
        #self.load_model_weights("Rollout_10")
        #print("loaded model weights")
            
        self.test_run_comparision=[]
        for test_run in range(len(test_keys)):
            test_bar.update()
            key_test=str(test_keys[test_run])
        
            
            ############# locally restrict the data for the training - we are restricting the number of timesteps to num_timesteps######
            negative_vals=[]
            for i in [0,1]:
                negative_vals.append(list(filter(lambda x: x < 1, data_irl[key_test][i])))

            num_timesteps=min(self.min_timesteps,len(negative_vals[0]),len(negative_vals[1]))
            
            # Ais initialization
            time_step_loss=0
            ais=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device)
            action_list=[0,0]             

            time_step_comparision=[]
            for time_step in range(num_timesteps):                    
                observation_list=[]
                for i in range(len(data_irl[key_test])-2): # exclude the last two columns (actions/inputs)
                    temp=data_irl[key_test][i][time_step]
                    if temp!= None:
                        observation_list.append(temp)
                    
                input_list=[]
                input_list.extend(observation_list)
                input_list.extend(action_list)
                input_tensor=torch.tensor(input_list,dtype=torch.float).to(self.device)
                print("\n")
                print("Before rollout, the input is ",input_tensor)
                # Update action list for the next time step and for the reference
                for i in [4,5]:
                    temp=data_irl[key_test][i][time_step]
                    if temp!= None:
                        action_list[i-4]=temp
                    else:
                        action_list[i-4]=0

                reference_list=[]
                observation_list_ref=[]
                for i in [1,3]:#range(len(data_irl[key_test])-2): # exclude the last two columns (actions/inputs)
                    temp=data_irl[key_test][i][time_step+1]
                    if temp!= None:
                        observation_list_ref.append(temp)                      
                reference_list.extend(observation_list_ref)
                reference_list.extend([action_list[1]])

                reference_tensor=torch.tensor(reference_list,dtype=torch.float).to(self.device)

                ais_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device) # Just initialize it to 0.5
                ais_rollout_dec=0.5*torch.ones(self.n_state_dec,dtype=torch.float).to(self.device) # Just initialize it to 0.5


                rollout_reference=0.5*torch.ones(self.n_output,dtype=torch.float).to(self.device)

                rollout_prediction=0.5*torch.ones(self.n_output,dtype=torch.float).to(self.device)

                internal_rollout_horizon=min(self.rollout,len(data_irl[key_test][0])-time_step-2)
                rollout_loss=0

                rollout_comparison=[]
                for k in range(internal_rollout_horizon):
                    temp_comparision=[]
                    
                    #print("\n")
                    if k==0:
                        rollout_input=input_tensor
                        rollout_reference=reference_tensor
                        ais_rollout_enc=ais
                        ais_rollout_enc=self.gen_model(rollout_input,ais_rollout_enc)
                        ais=ais_rollout_enc
                        
                        #decoder_input=list(ais_rollout_enc.detach().cpu().numpy())
                        #decoder_input.extend([time_step+k])
                        #decoder_input_tensor=torch.tensor(decoder_input,dtype=torch.float).to(self.device)
                        
                        time_step_tensor=torch.tensor([time_step+k],dtype=torch.float).to(self.device)
                        decoder_input_tensor=torch.cat((ais_rollout_enc,time_step_tensor),0)
                        rollout_prediction,ais_rollout_dec=self.pred_model(decoder_input_tensor,ais_rollout_dec)
                        
                        print("for rollout",k,"rollout_reference is ",rollout_reference)
                        print("for rollout",k,"decoder_input  is ",decoder_input_tensor,time_step+k)
                        print("for rollout",k,"rollout_prediction is ",rollout_prediction)
                        
                        rollout_loss+=-torch.distributions.MultivariateNormal(rollout_prediction,torch.eye(rollout_prediction.shape[0]).to(self.device)).log_prob(rollout_reference)

                        temp_comparision.extend(rollout_reference.detach().cpu().numpy())
                        temp_comparision.extend(rollout_prediction.detach().cpu().numpy())

                    else:
                        rollout_reference_list=[]                           
                        for i in [1,3,5]:#range(len(data_irl[key_test])):
                            if i<4:
                                rollout_reference_list.append(data_irl[key_test][i][time_step+k+1])
                            elif i>=4 and data_irl[key_test][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                rollout_reference_list.append(data_irl[key_test][i][time_step+k])
                            else:
                                rollout_reference_list.append(0) # Zero padding for the input action

                        rollout_reference=torch.tensor(rollout_reference_list,dtype=torch.float).to(self.device)

                        
                        #decoder_input=list(ais_rollout_enc.detach().cpu().numpy())
                        #decoder_input.extend([time_step+k])
                        #decoder_input_tensor=torch.tensor(decoder_input,dtype=torch.float).to(self.device)

                        time_step_tensor=torch.tensor([time_step+k],dtype=torch.float).to(self.device)
                        decoder_input_tensor=torch.cat((ais_rollout_enc,time_step_tensor),0)
                        rollout_prediction,ais_rollout_dec=self.pred_model(decoder_input_tensor,ais_rollout_dec)                                                
                        
                        print("for rollout",k,"rollout_reference is ",rollout_reference)
                        print("for rollout",k,"decoder_input  is ",decoder_input_tensor,time_step+k)
                        print("for rollout",k,"rollout_prediction is ",rollout_prediction)

                        temp_comparision.extend(rollout_reference.detach().cpu().numpy())
                        temp_comparision.extend(rollout_prediction.detach().cpu().numpy())

                        rollout_loss+=-torch.distributions.MultivariateNormal(rollout_prediction,torch.eye(rollout_prediction.shape[0]).to(self.device)).log_prob(rollout_reference)
                    rollout_comparison.append(temp_comparision)
                time_step_comparision.append(rollout_comparison)    
            self.test_run_comparision.extend(time_step_comparision)
    
        return self

In [94]:
####### Parameters #######
n_epochs=60
#min_timesteps=20
#n_rollout=10
#n_state_enc =24
#n_state_dec =12
min_timesteps=10
n_rollout=5
n_test=1
n_input=6
n_output=3
n_state_enc =4
n_state_dec =5 
learning_rate=0.0005
gen_model=2
pred_model=2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
####### Initialise the neural network #######

network=NN_structure(n_epochs,min_timesteps,n_rollout,n_test,n_input,n_output,n_state_enc,n_state_dec,learning_rate,gen_model,pred_model,device)
print("Training has started")
network.train()


In [105]:
# Test the network

####### Load the neural network weights from saved models #######

loaded_network=NN_structure(n_epochs,min_timesteps,n_rollout,n_test,n_input,n_output,n_state_enc,n_state_dec,learning_rate,gen_model,pred_model,device)
loaded_network.load_model_weights("reduced_timesteps10_Rollout_5_added_time_step_to_decoder_input")


test_keys=[keys_for_testing[15]]
test_results=loaded_network.test(test_keys)

0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00




Before rollout, the input is  tensor([-50.0000, -60.5453,  10.0000,   9.3482,   0.0000,   0.0000],
       device='cuda:0')
for rollout 0 rollout_reference is  tensor([-58.6157,   9.9482,   3.0000], device='cuda:0')
for rollout 0 decoder_input  is  tensor([1.0000, 1.0000, 1.0000, 1.0000, 0.0000], device='cuda:0',
       grad_fn=<CatBackward0>) 0
for rollout 0 rollout_prediction is  tensor([-41.4011,   9.0604,   0.1079], device='cuda:0', grad_fn=<AddBackward0>)
for rollout 1 rollout_reference is  tensor([-56.5660,  10.5482,   3.0000], device='cuda:0')
for rollout 1 decoder_input  is  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000], device='cuda:0',
       grad_fn=<CatBackward0>) 1
for rollout 1 rollout_prediction is  tensor([-41.4045,   9.0611,   0.1079], device='cuda:0', grad_fn=<AddBackward0>)
for rollout 2 rollout_reference is  tensor([-54.3964,  11.1482,   3.0000], device='cuda:0')
for rollout 2 decoder_input  is  tensor([1.0000, 1.0000, 1.0000, 1.0000, 2.0000], device='cuda:0',
  

In [103]:
test_results.test_run_comparision

[[[-58.615685, 9.948225, 3.0, -41.40107, 9.060353, 0.10786998],
  [-56.56604, 10.548224, 3.0, -41.40448, 9.06115, 0.1079319],
  [-54.396393, 11.148225, 3.0, -41.404797, 9.061213, 0.10793239],
  [-52.142635, 11.38936, 1.2056793, -41.404884, 9.061232, 0.10793246],
  [-49.857746, 11.459533, 0.35085994, -41.404926, 9.061241, 0.10793276]],
 [[-56.56604, 10.548224, 3.0, -41.40306, 9.060805, 0.10789752],
  [-54.396393, 11.148225, 3.0, -41.40479, 9.061211, 0.10793215],
  [-52.142635, 11.38936, 1.2056793, -41.404884, 9.061232, 0.10793246],
  [-49.857746, 11.459533, 0.35085994, -41.404926, 9.061241, 0.10793276],
  [-47.564804, 11.46989, 0.051787715, -41.404945, 9.061247, 0.107933074]],
 [[-54.396393, 11.148225, 3.0, -41.403942, 9.061008, 0.10791166],
  [-52.142635, 11.38936, 1.2056793, -41.404884, 9.061232, 0.10793242],
  [-49.857746, 11.459533, 0.35085994, -41.404926, 9.061241, 0.10793276],
  [-47.564804, 11.46989, 0.051787715, -41.404945, 9.061247, 0.107933074],
  [-45.271965, 11.458492, -0.05

In [86]:
#print(len(test_results.test_run_comparision[0][1]))

#test_results.test_run_comparision[19][:]


# Plot the results only for the HDV
pos_t=[]
vel_t=[]
acc_t=[]
pos_tref=[]
vel_tref=[]
acc_tref=[]
pos_t_hdv=[]
vel_t_hdv=[]
acc_t_hdv=[]
pos_tref_hdv=[]
vel_tref_hdv=[]
acc_tref_hdv=[]

# set which timestep to plot
time_step=10

for index in range(len(test_results.test_run_comparision[time_step])):
    '''
    pos_tref.append(test_results.test_run_comparision[time_step][index][0])
    vel_tref.append(test_results.test_run_comparision[time_step][index][2])
    acc_tref.append(test_results.test_run_comparision[time_step][index][4])
    pos_t.append(test_results.test_run_comparision[time_step][index][6])
    vel_t.append(test_results.test_run_comparision[time_step][index][8])
    acc_t.append(test_results.test_run_comparision[time_step][index][10])
    
    #pos_t_hdv.append(test_results.test_run_comparision[time_step][index][1])
    #vel_t_hdv.append(test_results.test_run_comparision[time_step][index][3])
    #acc_t_hdv.append(test_results.test_run_comparision[time_step][index][5])
    #pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][7])
    #vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][9])
    #acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][11])

    
    pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][1])
    vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][3])
    acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][5])
    pos_t_hdv.append(test_results.test_run_comparision[time_step][index][7])
    vel_t_hdv.append(test_results.test_run_comparision[time_step][index][9])
    acc_t_hdv.append(test_results.test_run_comparision[time_step][index][11])
    '''

    pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][0])
    vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][1])
    acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][2])
    pos_t_hdv.append(test_results.test_run_comparision[time_step][index][3])
    vel_t_hdv.append(test_results.test_run_comparision[time_step][index][4])
    acc_t_hdv.append(test_results.test_run_comparision[time_step][index][5])


IndexError: list index out of range

In [ ]:
plot_data=data_irl[str(keys_for_testing[10])]    

pos_t_hdv=[]
vel_t_hdv=[]
acc_t_hdv=[]
pos_tref_hdv=[]
vel_tref_hdv=[]
acc_tref_hdv=[]

    for index in range(len(plot_data[0])):

        pos_t_hdv.append(test_results.test_run_comparision[time_step][index][1].detach().cpu().numpy())
        vel_t_hdv.append(test_results.test_run_comparision[time_step][index][3].detach().cpu().numpy())
        acc_t_hdv.append(test_results.test_run_comparision[time_step][index][5].detach().cpu().numpy())
        pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][7].detach().cpu().numpy())
        vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][9].detach().cpu().numpy())
        acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][11].detach().cpu().numpy())

In [29]:
# Plotting outputs #
import matplotlib.colors as mcolors
####### For X - lateral#########
fig, ax = plt.subplots()
#plt.plot(y_ref,x_ref,'r',linewidth=2.0,label="Actual")
#plt.plot(y_pred,x_pred,'b',linewidth=2.0,label="Predicted")
plt.figure(facecolor='white')
plt.plot(pos_t_hdv,mcolors.CSS4_COLORS['salmon'],linewidth=2.0,label="Actual")
plt.plot(pos_tref_hdv,mcolors.CSS4_COLORS['limegreen'],linewidth=2.0,label="Predicted")
plt.xlabel('Time in (1/10)s')
plt.ylabel('Position along the road m')
plt.title('Comparision_of_position_along_road')
plt.legend()
plt.grid()
plt.savefig('graphs/irl_graphs/Comparision_rollout10_timestep_min20_posref_pospred_g1_p2_lr0005.png')
plt.close()

fig, ax = plt.subplots()
plt.figure(facecolor='white')
plt.plot(vel_t_hdv,mcolors.CSS4_COLORS['salmon'],linewidth=2.0,label="Actual")
plt.plot(vel_tref_hdv,mcolors.CSS4_COLORS['limegreen'],linewidth=2.0,label="Predicted")
plt.xlabel('Time in (1/10)s')
plt.ylabel('Velocity along the road in m/s')
plt.title('Comparision_Lateral_Velocities')
plt.legend()
plt.grid()
plt.savefig('graphs/irl_graphs/Comparision_rollout10_timestep_min20_vref_vpre_g1_p2_lr0005.png')
plt.close()

In [78]:
test_results.test_run_comparision[1][0]
#len(test_results.test_run_comparision)

[tensor(-46.4000, device='cuda:0'),
 tensor(-53.2063, device='cuda:0'),
 tensor(8., device='cuda:0'),
 tensor(7.4680, device='cuda:0'),
 tensor(-5., device='cuda:0'),
 tensor(-1.3502, device='cuda:0'),
 tensor(-31.8437, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(-27.8377, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(4.1773, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(9.0293, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(-1.2700, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(0.0087, device='cuda:0', grad_fn=<UnbindBackward0>)]

In [50]:
a=torch.tensor([1,2,3],dtype=torch.float)
b=torch.tensor([4],dtype=torch.float)
c=torch.cat((a,b),0)